In [9]:
import qlib
from qlib.data import D
from qlib.data.dataset.handler import DataHandlerLP
from qlib.data.data import Cal
from qlib.data.filter import NameDFilter
import pandas as pd
import os
import subprocess
import requests
import json
import time

In [3]:
'''
从DW（数仓）中下载数据，1min，1day，根据需求自行下载，一般下载一次即可
'''
# 设置token
headers = {"token": "wanglei@hztantra.com.cn"}  # your token

In [4]:
# 把返回的json格式数据解析程dataframe格式，retCode=200表示成功，其他值均表示失败，失败原因查看retMsg
def parse_json_data_to_dataframe(ret_data: None):
    bl = True
    msg = None
    df_lt = []
    if ret_data["retCode"] == 200:
        data_json = json.loads(ret_data["retData"])
        # 将提取的结果转换为DataFrame
        if "columns" in data_json.keys():
            df = pd.DataFrame.from_dict(data_json["data"])
            df.columns = data_json["columns"]
            df_lt.append(df)
        else:
            for key in data_json.keys():
                print(key)
                json_data = json.loads(data_json[key])
                if "data" in json_data.keys():
                    df = pd.DataFrame.from_dict(json_data["data"])
                    df.columns = json_data["columns"]
                    df_lt.append(df)
                else:
                    for ky in json_data.keys():
                        try:
                            j_data = json.loads(json_data[ky])
                            df = pd.DataFrame.from_dict(j_data["data"])
                            df.columns = j_data["columns"]
                            df_lt.append(df)
                        except:
                            # 再深层次自行解析
                            pass
    else:
        msg = ret_data["retMsg"]
        bl = False
    return bl, msg, df_lt


# 上传/请求数据函数
def post_data(url: str = None, datas: dict = None):
    r = requests.post(url=url, headers=headers, data=json.dumps(datas))
    return r.json()


# 获取数据函数
def get_data(url: str = None):
    r = requests.get(url=url, headers=headers)
    return r.json()

In [8]:
'''
其中
tr_code: 符合000001.SZ格式
start_time: 开始时间，格式为YYYYMMDDHHMMSS
end_time: 结束时间，格式为YYYYMMDDHHMMSS
k_type: 数据类型，1min表示1分钟数据，1day表示1天数据
adj_type: 复权类型，f表示前复权，b表示后复权
'''

# 只取可转债数据
code_table_df = pd.read_csv('D:/data/STKInfo/BOND_ConvertInfo.csv')
cb_code_list = code_table_df['SYMBOL9'].tolist()

# 重试参数
max_retries = 5
backoff_factor = 1  # 基础等待秒数，指数退避会以此为基数

for code in cb_code_list:
    print(f'正在处理：{code}...')
    url = f"http://hztantra.synology.me:8000/v1/api/market/get_history.json?tr_code={code}&start_time=20220101&end_time=20250707&k_type=1day&adj_type=f"

    # 带重试的请求，避免 ConnectionResetError 导致整个流程中断
    ret_data = None
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, headers=headers, timeout=15)
            resp.raise_for_status()
            ret_data = resp.json()
            break
        except requests.exceptions.RequestException as e:
            print(f"请求失败（{code}）attempt {attempt+1}/{max_retries}: {e}")
            if attempt < max_retries - 1:
                sleep_time = backoff_factor * (2 ** attempt)
                print(f"等待 {sleep_time} 秒后重试...")
                time.sleep(sleep_time)
            else:
                print(f"达到最大重试次数，跳过 {code}")
    if ret_data is None:
        # 未获取到数据则跳过当前 code
        continue

    # 返回数据解析成DataFrame
    try:
        bl, msg, df_lt = parse_json_data_to_dataframe(ret_data=ret_data)
        if bl:
            for df in df_lt:
                df.to_csv(f'D:/data/day1_data_raw/{code}.csv', index=False)
        else:
            print(msg)
    except Exception as ex:
        print(f'数据解析失败（{code}）: {ex}')
        continue

正在处理：110025.SH...
请求失败（110025.SH）attempt 1/5: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
等待 1 秒后重试...
请求失败（110025.SH）attempt 2/5: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
等待 2 秒后重试...
请求失败（110025.SH）attempt 3/5: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
等待 4 秒后重试...
请求失败（110025.SH）attempt 4/5: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
等待 8 秒后重试...
请求失败（110025.SH）attempt 5/5: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
达到最大重试次数，跳过 110025.SH
正在处理：110027.SH...
请求失败（110027.SH）attempt 1/5: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
等待 1 秒后重试...
请求失败（110027.SH）attempt 2/5: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
等待 2 秒后重试...
请求失败（110027.SH）at

KeyboardInterrupt: 

In [ ]:
'''
# 将原始数据转换成中间数据（csv）
'''
features = ['datetime','instrument','open','high','low','close','volume','amount']#,'marketcap','new','ST','CONVERTIBLEPREMIUMRATE']#,'pre_close','TURNOVERRATE','new','ST']
for csvname in os.listdir('min1_data_raw'):
    instrument = csvname[:9]
    print(f"正在转换中间文件：{instrument}...")
    df_tmp = pd.read_csv(f'min1_data_raw/{instrument}.csv')
    df_tmp['datetime'] = pd.to_datetime(df_tmp['date'] * 10000 + df_tmp['time'], format='%Y%m%d%H%M')#pd.to_datetime(df_tmp['date'], format='%Y-%m-%d')
    df_tmp['instrument'] = instrument
    df_tmp = df_tmp[features]
    df_tmp.set_index(['datetime', 'instrument'], inplace=True)
    df_tmp.to_csv(f'min1_data_mid/{instrument}.csv')

正在转换中间文件：110034.SH...
正在转换中间文件：110038.SH...
正在转换中间文件：110043.SH...
正在转换中间文件：110044.SH...
正在转换中间文件：110045.SH...
正在转换中间文件：110047.SH...
正在转换中间文件：110048.SH...
正在转换中间文件：110052.SH...
正在转换中间文件：110053.SH...
正在转换中间文件：110055.SH...
正在转换中间文件：110056.SH...
正在转换中间文件：110057.SH...
正在转换中间文件：110058.SH...
正在转换中间文件：110059.SH...
正在转换中间文件：110060.SH...
正在转换中间文件：110061.SH...
正在转换中间文件：110062.SH...
正在转换中间文件：110063.SH...
正在转换中间文件：110064.SH...
正在转换中间文件：110066.SH...
正在转换中间文件：110067.SH...
正在转换中间文件：110068.SH...
正在转换中间文件：110070.SH...
正在转换中间文件：110071.SH...
正在转换中间文件：110072.SH...
正在转换中间文件：110073.SH...
正在转换中间文件：110074.SH...
正在转换中间文件：110075.SH...
正在转换中间文件：110076.SH...
正在转换中间文件：110077.SH...
正在转换中间文件：110079.SH...
正在转换中间文件：110080.SH...
正在转换中间文件：110081.SH...
正在转换中间文件：110082.SH...
正在转换中间文件：110083.SH...
正在转换中间文件：110084.SH...
正在转换中间文件：110085.SH...
正在转换中间文件：110086.SH...
正在转换中间文件：110087.SH...
正在转换中间文件：110088.SH...
正在转换中间文件：110089.SH...
正在转换中间文件：110090.SH...
正在转换中间文件：110091.SH...
正在转换中间文件：110092.SH...
正在转换中间文件：110093.SH...
正在转换中间文件：1

In [ ]:
'''
csv转bin
其中
E:/PY_qlib_env/.venv/Scripts/python.exe 指python路径
E:/PY_qlib_env/.venv/Lib/site-packages/qlib/scripts/dump_bin.py dump_all 指qlib的dump_bin.py路径
--csv_path 原始数据路径
--qlib_dir 转换后数据路径
--freq 1min 转换后数据频率
--date_field_name datetime 转换后数据时间字段
--symbol_field_name instrument 转换后数据股票代码字段
--include_fields open,high,low,close,volume 转换后数据包含字段
'''
print(f"正在执行把csv文件转换为bin文件...")
subprocess.run(f'D:/anaconda/envs/py10/python.exe C:/Users/tantra/Desktop/data/.venv/Lib/site-packages/qlib/scripts/dump_bin.py dump_all --csv_path min1_data_mid --qlib_dir min1_data_qlib --freq 1min --date_field_name datetime --symbol_field_name instrument --include_fields open,high,low,close,volume,amount')#,marketcap,new,ST,CONVERTIBLEPREMIUMRATE')#pre_close,TURNOVERRATE,new,ST')

正在执行把csv文件转换为bin文件...


CompletedProcess(args='D:/anaconda/envs/py10/python.exe C:/Users/tantra/Desktop/data/.venv/Lib/site-packages/qlib/scripts/dump_bin.py dump_all --csv_path min1_data_mid --qlib_dir min1_data_qlib --freq 1min --date_field_name datetime --symbol_field_name instrument --include_fields open,high,low,close,volume,amount', returncode=0)